In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import scarches

from anndata import AnnData
from scarches.models.scpoli import scPoli

import umap

INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)
 captum (see https://github.com/pytorch/captum).


In [2]:
import torch
print(torch.cuda.is_available())

False


## Specify working directory

In [3]:
WD = "/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93vel/atlas_building"
VERSION = 21

In [4]:
out_folder = f"{WD}/q2r_models/"
model_path = f"{out_folder}/model_v{VERSION}"

## Functions

In [5]:
def read_samples(project_dir, sample_sheet, annot_dir):
    adata_list=[]
    for sample in sample_sheet.sample_id:
        sample_file=f"{project_dir}/h5ad_pyraw/cp_h5ad/{sample}_pyraw.h5ad"
        adata = sc.read_h5ad(sample_file)
        adata.obs['sample_id'] = sample
        origin = sample_sheet[sample_sheet['sample_id'] == sample]['derive'].values[0]
        adata.obs['derive'] = origin
        adata.var = adata.var.drop(adata.var.columns, axis=1)
        meta_file = f"{annot_dir}/{sample}_annotation.txt"
        if os.path.exists(meta_file) and os.path.getsize(meta_file)>1:
            meta_data = pd.read_csv(meta_file, sep="\t", index_col=0)
            adata.obs = pd.merge(adata.obs, meta_data, left_index=True, right_index=True)
        else:
            adata.obs['level_1'] = 'epithelial'
            adata.obs['level_2'] = 'na'

        adata_list.append(adata)
    adata_concat = anndata.concat(adata_list, join='outer', fill_value=0)
    adata_concat.obs['publication'] = ['_'.join(i.split('_')[:3]) for i in adata_concat.obs.sample_id.tolist()]

    adata_concat.obs.index.name = "cells"

    return adata_concat

def get_counts(adata):
    t = adata.X.toarray()
    data_df = pd.DataFrame(data=t, index=adata.obs_names, columns=adata.var_names)
    data_df = np.expm1(data_df)
    counts_df = data_df.T.mul(adata.obs.n_counts).div(10000)
    counts_df = counts_df.T.iloc[:,:]
    counts_df = counts_df.round(0).astype(np.float32)
    return counts_df
    
def clear_genes(project_dir, adata):
    # clear_genes = pd.read_csv("/home/xuq44/refgenomes/hg38/hg_genes_clear_nocc.txt", header=None)[0].tolist()
    clear_genes = pd.read_csv(os.path.join(project_dir, "hg_genes_clear.txt"), header=None)[0].tolist()
    sub_clear_genes = [i for i in clear_genes if i in adata.var.index.tolist()]
    adata = adata[:, sub_clear_genes]
    
    return adata

def pre_inti0(adata):
    adata.layers['counts']=adata.X
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata

    return adata

def get_kwargs():
    early_stopping_kwargs = {
        "early_stopping_metric": "val_prototype_loss",
        "mode": "min",
        "threshold": 0,
        "patience": 20,
        "reduce_lr": True,
        "lr_patience": 13,
        "lr_factor": 0.1,
    }
    return early_stopping_kwargs

def train_scpoli(adata):
    early_stopping_kwargs = get_kwargs()

    scpoli_model = scPoli(
        adata=adata,
        unknown_ct_names=['na'],
        condition_keys=['sample_id'],
        cell_type_keys=['level_2'],
        embedding_dims=10,  # default
        latent_dim=10,  # default
        hidden_layer_sizes=[512, 512],
        recon_loss='nb',
    )

    scpoli_model.train(
        n_epochs=50,
        pretraining_epochs=4,
        early_stopping_kwargs=early_stopping_kwargs,
        eta=10,
        alpha_epoch_anneal=100,
    )

    return scpoli_model

def map_query(adata, model):
    early_stopping_kwargs = get_kwargs()

    scpoli_query = scPoli.load_query_data(
        adata=adata,
        reference_model=model,
        labeled_indices=[],
    )

    scpoli_query.train(
        n_epochs=200,
        pretraining_epochs=80,
        early_stopping_kwargs=early_stopping_kwargs,
        eta=10,
        alpha_epoch_anneal=100,
    )

    return scpoli_query

def umap_transform(adata_ref, adata_que, cell_type_col):
    model = umap.UMAP(n_neighbors=5, random_state=42, min_dist=0.5).fit(adata_ref.X)
    adata_ref.obsm['X_umap'] = model.transform(adata_ref.X)
    adata_que.obsm['X_umap'] = model.transform(adata_que.X)
    
    adata_ref.obs['query'] = 0
    adata_que.obs['query'] = 1
    
    adata_ref.obs['cell_type_pred'] = np.nan
    
    # harmonized annotation: reference labels transferred to query cells
    adata_ref.obs['cell_type'] = adata_ref.obs['level_2'].copy()
    adata_que.obs['cell_type'] = adata_que.obs['cell_type_pred'].copy()
    
    # original reference cell annotation
    adata_ref.obs['cell_type_ref'] = adata_ref.obs['level_2'].copy()
    adata_que.obs['cell_type_ref'] = np.nan
    
    # original query cell annotation
    # adata_ref.obs['cell_type_que'] = np.nan
    # adata_que.obs['cell_type_que'] = adata_que.obs['level_2'].copy()
    
    adata_all = anndata.concat([adata_ref, adata_que])

    return adata_all


## Load samples

In [6]:
project_dir = os.path.join(WD, 'data')
annot_dir = os.path.join(project_dir, 'sample_annot6')
sample_sheet = pd.read_csv(os.path.join(project_dir, 'all_samples_sheets.txt'), sep='\t')
sample_sheet = sample_sheet[sample_sheet.tissue=='lung']

### Load reference data

In [7]:
# FETAL
cell_type_col = 'new_celltype'
fetal = sc.read(os.path.join(project_dir, 'reference/Assembled10DomainsEpithelial.h5ad'))
fetal = clear_genes(project_dir, fetal)
fetal.obs['level_1'] = 'epithelial'
fetal.obs['level_2'] = fetal.obs[cell_type_col].copy()
fetal.obs['sample_id'] = fetal.obs['donor'].copy()
fetal.obs['maturity'] = 'fetal'

/tmp/ipykernel_380354/3250957859.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  fetal.obs['level_1'] = 'epithelial'


In [8]:
# revert the normalized, transformed data back to counts
fetal_counts = get_counts(fetal)
fetal.X = fetal_counts.values.copy()

In [9]:
print(np.max(fetal.X))
print(np.min(fetal.X))
print(type(fetal.X[0,0]))

11123.0
0.0
<class 'numpy.float32'>


In [10]:
# MATURE
cell_type_col = 'predicted_labels'
mature = sc.read(os.path.join(project_dir, 'reference/Barbry_Leroy_2020_epithelial_annot.h5ad'))
mature = clear_genes(project_dir, mature)
mature.obs['level_1'] = 'epithelial'
mature.obs['level_2'] = mature.obs[cell_type_col].copy()
mature.obs['sample_id'] = mature.obs['sample'].copy()
mature.obs['maturity'] = 'mature'
mature.uns['log1p'] = {'base': None}

/tmp/ipykernel_380354/1559326118.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  mature.obs['level_1'] = 'epithelial'


In [11]:
mature.X = mature.layers['counts'].copy()

In [12]:
print(np.max(mature.X))
print(np.min(mature.X))
print(type(mature.X[0,0]))

35330.0
0.0
<class 'numpy.float32'>


In [13]:
reference = anndata.concat([fetal, mature])

/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/anndata/_core/merge.py:1111: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(


In [14]:
reference.obs['derive'] = np.nan
reference.obs

,level_1,level_2,sample_id,maturity,derive
ACACCCTAGGTAAACT-5891STDY8062349,epithelial,Ciliated,HDBR14808,fetal,NaN
ACAGCTAAGGGTTTCT-5891STDY8062349,epithelial,Late tip,HDBR14808,fetal,NaN
ACCCACTGTATGAAAC-5891STDY8062349,epithelial,Club,HDBR14808,fetal,NaN
ACGAGGAGTACCGCTG-5891STDY8062349,epithelial,MUC16+ ciliated,HDBR14808,fetal,NaN
ACGAGGAGTCCTGCTT-5891STDY8062349,epithelial,Ciliated,HDBR14808,fetal,NaN
...,...,...,...,...,...
TTTCCTCGTCAGTGGA-1-34,epithelial,Suprabasal,34,mature,NaN
TTTGGTTCAGTCGATT-1-34,epithelial,Basal resting,34,mature,NaN
TTTGGTTCATTTCAGG-1-34,epithelial,Suprabasal,34,mature,NaN
TTTGTCAAGTTCCACA-1-34,epithelial,Club (non-nasal),34,mature,NaN


In [15]:
sc.pp.highly_variable_genes(reference,
                            n_top_genes=3000, 
                            batch_key='sample_id',
                            flavor='seurat_v3')

reference = reference[:,reference.var.highly_variable]

### Load organoid data

In [16]:
organoid = read_samples(project_dir, sample_sheet, annot_dir)

/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/anndata/_core/merge.py:1111: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(
/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [17]:
print(np.max(organoid.X))
print(np.min(organoid.X))
print(type(organoid.X[0,0]))

25001.0
0.0
<class 'numpy.float64'>


In [18]:
organoid.X = organoid.X.astype(np.float32)
print(type(organoid.X[0,0]))

<class 'numpy.float32'>


In [19]:
organoid.obs.level_1.value_counts()

epithelial    225487
Name: level_1, dtype: int64

In [20]:
organoid.obs.level_2.value_counts()

alveolar type 2 (AT2) cells    61221
basal cells                    32813
club cells                     30613
goblet cells                   27694
stem cells                     19224
na                             16034
alveolar type 1 (AT1) cells    15337
neuroendocrine cells           14272
airway secretory cells          5626
ciliated cells                  2653
Name: level_2, dtype: int64

In [21]:
organoid.obs['orig_cell_types'] = organoid.obs['level_2'].copy()
organoid.obs['level_2'] = 'na'

In [22]:
organoid.obs

,initial_size_spliced,initial_size_unspliced,initial_size,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo,total_counts_hb,...,n_genes,sample_id,derive,level_1,level_2,level_3,Cell_type,n_counts,publication,orig_cell_types
cells,,,,,,,,,,,,,,,,,,,,,
AAACCCACAAAGCTAA,31397.0,14368.0,31397.0,6808,39508.0,4868.0,12.321555,7983.0,20.206034,1.0,...,6808.0,Chan_NatCommun_2022_bronchial_organoids,ASC,epithelial,na,basal cells,NaN,NaN,Chan_NatCommun_2022,basal cells
AAACCCACACTGTGTA,24573.0,8025.0,24573.0,4675,31483.0,3729.0,11.844487,7484.0,23.771559,0.0,...,4675.0,Chan_NatCommun_2022_bronchial_organoids,ASC,epithelial,na,club cells,NaN,NaN,Chan_NatCommun_2022,club cells
AAACCCATCCGCAGTG,21147.0,6246.0,21147.0,4822,26328.0,2810.0,10.673048,5713.0,21.699331,2.0,...,4822.0,Chan_NatCommun_2022_bronchial_organoids,ASC,epithelial,na,club cells,NaN,NaN,Chan_NatCommun_2022,club cells
AAACCCATCTAGTGTG,8687.0,3785.0,8687.0,2726,11182.0,632.0,5.651941,3580.0,32.015739,0.0,...,2726.0,Chan_NatCommun_2022_bronchial_organoids,ASC,epithelial,na,basal cells,NaN,NaN,Chan_NatCommun_2022,basal cells
AAACGAACAACTCCCT,11375.0,5798.0,11375.0,3956,14544.0,1113.0,7.652640,4326.0,29.744226,0.0,...,3956.0,Chan_NatCommun_2022_bronchial_organoids,ASC,epithelial,na,basal cells,NaN,NaN,Chan_NatCommun_2022,basal cells
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGATCAGTCTAGCT,9504.0,21130.0,9504.0,4459,11821.0,2224.0,18.813974,739.0,6.251586,0.0,...,4459.0,Hein_Dev_2022_Spheroids,IPS,epithelial,na,NaN,NaN,NaN,Hein_Dev_2022,na
TTTGATCCAACATACC,50587.0,52023.0,50587.0,8910,61103.0,1360.0,2.225750,11083.0,18.138226,5.0,...,8910.0,Hein_Dev_2022_Spheroids,IPS,epithelial,na,NaN,NaN,NaN,Hein_Dev_2022,na
TTTGGAGAGGAGAGGC,9358.0,9525.0,9358.0,3935,10915.0,2025.0,18.552450,1227.0,11.241410,1.0,...,3935.0,Hein_Dev_2022_Spheroids,IPS,epithelial,na,NaN,NaN,NaN,Hein_Dev_2022,na


In [23]:
# select only those highly variable genes that occur in both the reference and organoid data
try:
    organoid = organoid[:,reference.var.index]
except:
    a = set(reference.var.index)
    b = set(organoid.var.index)
    overlap = a.intersection(b)
    reference = reference[:,pd.Index(overlap)]
    organoid = organoid[:,pd.Index(overlap)]

In [24]:
print(reference.shape)
print(organoid.shape)

(86830, 2989)
(225487, 2989)


## Integrate reference data

In [25]:
# DO NOT RUN IF YOU HAVE TRAINED A MODEL BEFORE!
scpoli_model = train_scpoli(reference)

/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/scarches/models/scpoli/scpoli_model.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self.adata.obs['conditions_combined'] = adata.obs[condition_keys].apply(lambda x: '_'.join(x))


Embedding dictionary:
 	Num conditions: [47]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 2989 512 10
	Hidden Layer 1 in/out: 512 512
	Mean/Var Layer in/out: 512 10
Decoder Architecture:
	First Layer in, out and cond:  10 512 10
	Hidden Layer 1 in/out: 512 512
	Output Layer in/out:  512 2989 

Initializing dataloaders
Starting training
 |█-------------------| 8.0%  - val_loss:  866.27 - val_cvae_loss:  866.27
Initializing unlabeled prototypes with Leiden with an unknown number of  clusters.
Clustering succesful. Found 55 clusters.
 |████████████████████| 100.0%  - val_loss:  907.26 - val_cvae_loss:  885.00 - val_prototype_loss:   22.26 - val_unlabeled_loss:    1.58 - val_labeled_loss:    2.23


In [26]:
reference.obsm['X_scPoli'] = scpoli_model.get_latent(
    reference,
    mean=True,
)

/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/scarches/models/scpoli/scpoli_model.py:344: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  c = torch.tensor(label_tensor, device=device).T


In [27]:
# visualize the latent representation of reference cells computed by scPoli
sc.pp.neighbors(reference, use_rep='X_scPoli')
sc.tl.umap(reference)
sc.pl.umap(
    reference, 
    color=['maturity', 'level_2'],
    show=True,
    frameon=False,
    save=f'_scPoli_latent_v{VERSION}.png',
)

/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


In [28]:
scpoli_model.save(model_path)

## Load model

In [29]:
# load a Q2R model if you've trained one before
# scpoli_model = scarches.models.scpoli.scPoli.load(model_path, reference)

## Map organoid to reference cells

In [ ]:
scpoli_query = map_query(organoid, scpoli_model)

# get latent representation of reference data
scpoli_query.model.eval()
data_latent_source = scpoli_query.get_latent(
    reference,
    mean=True,
)

# get latent representation of query data
data_latent_target = scpoli_query.get_latent(
    organoid,
    mean=True,
)

adata_latent_source = AnnData(data_latent_source)
adata_latent_source.obs = reference.obs.copy()

adata_latent_target = AnnData(data_latent_target)
adata_latent_target.obs = organoid.obs.copy()

/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/scarches/models/scpoli/scpoli_model.py:766: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['conditions_combined'] = adata.obs[condition_keys].apply(lambda x: '_'.join(x))
/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/dss/dsshome1/06/di93vel/.conda/envs/heoa_venv/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Embedding dictionary:
 	Num conditions: [99]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 2989 512 10
	Hidden Layer 1 in/out: 512 512
	Mean/Var Layer in/out: 512 10
Decoder Architecture:
	First Layer in, out and cond:  10 512 10
	Hidden Layer 1 in/out: 512 512
	Output Layer in/out:  512 2989 

The missing labels are: {'na'}
Therefore integer value of those labels is set to -1
The missing labels are: {'na'}
Therefore integer value of those labels is set to -1
The missing labels are: {'na'}
Therefore integer value of those labels is set to -1
The missing labels are: {'na'}
Therefore integer value of those labels is set to -1
Initializing dataloaders
Starting training
 |████----------------| 23.0%  - val_loss: 1935.53 - val_cvae_loss: 1935.53

In [ ]:
# get label annotations
results_dict = scpoli_query.classify(organoid, scale_uncertainties=True)
adata_latent_target.obs['cell_type_pred'] = results_dict['level_2']['preds'].tolist()
adata_latent_target.obs['cell_type_uncert'] = results_dict['level_2']['uncert'].tolist()
adata_latent_target.obs['classifier_outcome'] = (
    adata_latent_target.obs['cell_type_pred'] == adata_latent_target.obs['level_2']
)

In [ ]:
adata_latent = umap_transform(adata_latent_source, adata_latent_target, cell_type_col=cell_type_col)

## Plots

In [ ]:
sc.pl.umap(
    adata_latent,
    color='cell_type',
    show=True,
    frameon=False,
    save=f'_new_celltype_v{VERSION}.png',
)

In [ ]:
sc.pl.umap(
    adata_latent,
    color='cell_type_ref',
    show=True,
    frameon=False,
)

In [ ]:
sc.pl.umap(
    adata_latent,
    color='query',
    show=True,
    frameon=False,
    save=f'_query_v{VERSION}.png',
)

In [ ]:
sc.pl.umap(
    adata_latent,
    color='cell_type_pred',
    show=True,
    frameon=False,
    save=f'_cell_type_pred_v{VERSION}.png',
)

In [ ]:
sc.pl.umap(
    adata_latent,
    color='derive',
    show=True,
    frameon=False,
    save=f'_derive_v{VERSION}.png',
)

In [ ]:
sc.pl.umap(
    adata_latent,
    color='cell_type_uncert',
    show=True,
    frameon=False,
    cmap='magma',
    # vmax=1,
    save=f'_uncert_v{VERSION}.png',
)

## Save latent representation

In [ ]:
adata_latent.write(os.path.join(WD, f"data/q2r_fetal_adata_latent_v{VERSION}.h5ad"))

In [ ]:
# uncertainty
# filtered data
# proliferation markers